# Basic Message - Alice

**Should be run alongside [Bob](http://localhost:8889/lab/tree/3%20Messages/Part%201%20-%20Basic%20Message.ipynb)**

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"


# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, api_key=API_KEY)

## Setup a listener for the basicmessages topic

This is emitted using PyPubSub when the controller receives a basicmessages webhook from the agent. This happens everytime the agent receives a basicmessage.

In [3]:
def messages_handler(payload):
    connection_id = payload["connection_id"]
    print("Handle message", payload, connection_id)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

In [4]:
def messages_handler2(payload):
    connection_id = payload["connection_id"]
    print("Second Handler", payload, connection_id)


message_listener2 = {
    "handler": messages_handler2,
    "topic": "basicmessages"
}

agent_controller.add_listener(message_listener2)

## Verify an Active Connection Exists

This should have been established through the setup docker container using the create_connection.py script.

In [5]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")
    

Results :  [{'rfc23_state': 'completed', 'their_role': 'inviter', 'request_id': '5d457ff6-28c5-429d-b7cd-e3ab3b0dc829', 'invitation_key': 'D6x1TzfFP76ihR9zRZFdUgL4mvcE2F8JKCroQJyno1SE', 'created_at': '2021-03-29 15:20:13.311490Z', 'their_did': 'Rn6qN2oFfgPyRKoYP2JuTe', 'their_label': 'Bob', 'invitation_mode': 'once', 'my_did': 'MrasHcPmBT3H8VVYiaR7eA', 'connection_id': 'd61e6495-1b0a-41de-b9e6-db799e6e8bdf', 'routing_state': 'none', 'updated_at': '2021-03-29 15:20:33.593695Z', 'state': 'active', 'accept': 'manual'}]
Connection : {'rfc23_state': 'completed', 'their_role': 'inviter', 'request_id': '5d457ff6-28c5-429d-b7cd-e3ab3b0dc829', 'invitation_key': 'D6x1TzfFP76ihR9zRZFdUgL4mvcE2F8JKCroQJyno1SE', 'created_at': '2021-03-29 15:20:13.311490Z', 'their_did': 'Rn6qN2oFfgPyRKoYP2JuTe', 'their_label': 'Bob', 'invitation_mode': 'once', 'my_did': 'MrasHcPmBT3H8VVYiaR7eA', 'connection_id': 'd61e6495-1b0a-41de-b9e6-db799e6e8bdf', 'routing_state': 'none', 'updated_at': '2021-03-29 15:20:33.59369

In [6]:
print(connection["connection_id"])
id = connection["connection_id"]

d61e6495-1b0a-41de-b9e6-db799e6e8bdf
WEBHOOK RECIEVED -  basicmessages
WEBHOOK RECIEVED -  basicmessages
HANDLE WEBHOOK - basicmessages
Handle message {'connection_id': 'd61e6495-1b0a-41de-b9e6-db799e6e8bdf', 'message_id': 'ee83bb20-c7ff-4961-9be2-a123eeff927c', 'content': 'Hello from Bob', 'state': 'received'} d61e6495-1b0a-41de-b9e6-db799e6e8bdf
Second Handler {'connection_id': 'd61e6495-1b0a-41de-b9e6-db799e6e8bdf', 'message_id': 'ee83bb20-c7ff-4961-9be2-a123eeff927c', 'content': 'Hello from Bob', 'state': 'received'} d61e6495-1b0a-41de-b9e6-db799e6e8bdf
PUBSUB SENT - basicmessages
HANDLE WEBHOOK - basicmessages
Handle message {'connection_id': 'd61e6495-1b0a-41de-b9e6-db799e6e8bdf', 'message_id': 'ee83bb20-c7ff-4961-9be2-a123eeff927c', 'content': 'Hello from Bob', 'state': 'received'} d61e6495-1b0a-41de-b9e6-db799e6e8bdf
Second Handler {'connection_id': 'd61e6495-1b0a-41de-b9e6-db799e6e8bdf', 'message_id': 'ee83bb20-c7ff-4961-9be2-a123eeff927c', 'content': 'Hello from Bob', 'state'

In [7]:
await agent_controller.messaging.trust_ping(id, "hello, world!")

{'thread_id': '5568ca7b-4922-4360-8794-dcb3910403dc'}

## Send a Basic Message over DIDComm to Bob

See [aries-rfc](https://github.com/hyperledger/aries-rfcs/tree/master/features/0095-basic-message)

You can send as messages as you want, if you are running the [basic-message tutorial on Bob's notebook](http://localhost:8889/lab/tree/3%20Messages/Part%201%20-%20Basic%20Message.ipynb) these will be received and printed by the message handler. You may have to run a code block to see the output.

In [12]:
basic_message = "hello from Alice"
response = await agent_controller.messaging.send_message(connection_id, basic_message)
print("BASIC MESSAGE - Alice -> Bob")
print(response)

BASIC MESSAGE - Alice -> Bob
{}
Handle message {'connection_id': '4f66d7ac-2771-49c7-bf5c-0b84a228cb54', 'message_id': '5e8795cf-5311-4166-ad5f-259f22c40d93', 'content': 'This is a response from Bob', 'state': 'received'} 4f66d7ac-2771-49c7-bf5c-0b84a228cb54
Handle message {'connection_id': '4f66d7ac-2771-49c7-bf5c-0b84a228cb54', 'message_id': '5e8795cf-5311-4166-ad5f-259f22c40d93', 'content': 'This is a response from Bob', 'state': 'received'} 4f66d7ac-2771-49c7-bf5c-0b84a228cb54
Handle message {'connection_id': '4f66d7ac-2771-49c7-bf5c-0b84a228cb54', 'message_id': '93ddc727-6b92-4acd-8e1b-eaea87ec1091', 'content': 'This is a response from Bob', 'state': 'received'} 4f66d7ac-2771-49c7-bf5c-0b84a228cb54
Handle message {'connection_id': '4f66d7ac-2771-49c7-bf5c-0b84a228cb54', 'message_id': '93ddc727-6b92-4acd-8e1b-eaea87ec1091', 'content': 'This is a response from Bob', 'state': 'received'} 4f66d7ac-2771-49c7-bf5c-0b84a228cb54


## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [ ]:
response = await agent_controller.terminate()
print(response)